<a href="https://colab.research.google.com/github/realBagher/sentiment_analysis_food_classifcation/blob/main/NLP_amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import gensim
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
train = pd.read_csv("/content/drive/MyDrive/Quera/Projects/Project 2/train_data.csv")

<ipython-input-3-143b7d2995a1>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("/content/drive/MyDrive/Quera/Projects/Project 2/train_data.csv")


In [4]:
train

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,2,NaN,False,2016-11-11,A2OSUEZJIN7BI,0511189877,NaN,Chris,I have an older URC-WR7 remote and thought thi...,Cannot Learn,1478822400
1,5,NaN,True,2016-06-06,A2NETQRG6JHIG7,0511189877,NaN,Qrysta White,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,1465171200
2,4,NaN,True,2016-03-10,A12JHGROAX49G7,0511189877,NaN,Linwood,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,1457568000
3,5,NaN,True,2016-01-14,A1KV65E2TMMG6F,0511189877,NaN,Dane Williams,I got tired of the remote being on the wrong s...,Same as TWC remote,1452729600
4,5,NaN,True,2016-10-20,A280POPEWI0NSA,0594459451,NaN,Kristina H.,After purchasing cheap cords from another webs...,Good Quality Cord,1476921600
...,...,...,...,...,...,...,...,...,...,...,...
838939,5,NaN,True,2017-07-25,A1OOVLE2KZ6KGA,B01HJCN1EI,NaN,Puddzee,These are my favorite charging cords for a few...,Worth the price.,1500940800
838940,1,NaN,True,2017-04-04,A77K1B31UAQ29,B01HJCN1EI,NaN,addictedtoreading,"Update....after 2 months of gentle use, cable ...",UPDATE...BREAKS AND SLOW CHARGING,1491264000
838941,3,NaN,True,2017-07-08,A2SVXUVUAWUDK2,B01HJH42KU,NaN,Andrew,These are okay. The connection becomes very if...,Hope this makes sense. You'd understand if you...,1499472000
838942,5,2.0,True,2016-12-01,A2HUZO7MQAY5I2,B01HJH40WU,NaN,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838944 entries, 0 to 838943
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         838944 non-null  int64 
 1   vote            191468 non-null  object
 2   verified        838944 non-null  bool  
 3   reviewTime      838944 non-null  object
 4   reviewerID      838944 non-null  object
 5   asin            838944 non-null  object
 6   style           490613 non-null  object
 7   reviewerName    838717 non-null  object
 8   reviewText      838944 non-null  object
 9   summary         838868 non-null  object
 10  unixReviewTime  838944 non-null  int64 
dtypes: bool(1), int64(2), object(8)
memory usage: 64.8+ MB


In [6]:
reviewText_corpus = ' '.join(train.reviewText)

In [7]:
reviewText_corpus[:2000]

"I have an older URC-WR7 remote and thought this would be an upgrade (and because TWC stuck me with it), but this one fails where the other one didn't.  The old remote could go head to head and learn.  I have 2 different Blu Ray players (LG & Panasonic) and this one fails on both.  It cannot learn buttons.  The biggest problem is in streaming when I need to hit the blue, red, yellow, green buttons.  These cannot work either of them, so I have to pull out the remote for that one setting.  I don't know why they give you multiple code methods but no learning. First time I've EVER had a remote that needed no programming whatsoever so I have to give it bragging rights.  All the other 8780Ls I've had certainly had to be set up first.\n\nYes I wish it was Silver or White or some color that would permit its buttons to be more easily read but its thinner in hand than the ATLAS OCAP 5-Device Cable Remote Control 1056B03 so its the best thing I can find for a Samsung DVR with Samsung TV on the TW

In [8]:
f = reviewText_corpus.replace("\n", " ")

In [9]:
f[:2000]

"I have an older URC-WR7 remote and thought this would be an upgrade (and because TWC stuck me with it), but this one fails where the other one didn't.  The old remote could go head to head and learn.  I have 2 different Blu Ray players (LG & Panasonic) and this one fails on both.  It cannot learn buttons.  The biggest problem is in streaming when I need to hit the blue, red, yellow, green buttons.  These cannot work either of them, so I have to pull out the remote for that one setting.  I don't know why they give you multiple code methods but no learning. First time I've EVER had a remote that needed no programming whatsoever so I have to give it bragging rights.  All the other 8780Ls I've had certainly had to be set up first.  Yes I wish it was Silver or White or some color that would permit its buttons to be more easily read but its thinner in hand than the ATLAS OCAP 5-Device Cable Remote Control 1056B03 so its the best thing I can find for a Samsung DVR with Samsung TV on the TWC 

In [10]:
len(f)

524916676

In [11]:
data = []

# iterate through each sentence in the file
for i in sent_tokenize(f):
    temp = []

    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())

    data.append(temp)

In [12]:
len(data)

6074937

In [ ]:
# Create CBOW model (Window size 5)
model = gensim.models.Word2Vec(data, min_count=1, vector_size=100, window=5)

In [13]:
# Create CBOW model (Window size 20)
model2 = gensim.models.Word2Vec(data, min_count=1, vector_size=100, window=20)

In [ ]:
model.save("/content/drive/MyDrive/Quera/Projects/Project 2/word2vec.model")

In [14]:
model2.save("/content/drive/MyDrive/Quera/Projects/Project 2/word2vec_2.model")

In [11]:
model = Word2Vec.load("/content/drive/MyDrive/Quera/Projects/Project 2/word2vec.model")

In [12]:
model2 = Word2Vec.load("/content/drive/MyDrive/Quera/Projects/Project 2/word2vec_2.model")

In [15]:
model.corpus_count

6074937

In [18]:
model.corpus_total_words

110147966

In [ ]:
vector = model.wv['guarantee']
vector

array([ 3.0777340e+00,  6.5127540e-01, -1.8482051e+00,  1.5112902e+00,
        3.6214793e-01, -2.3995836e+00, -2.5225635e+00, -8.8005608e-01,
        1.4434038e+00, -2.6636229e+00, -7.3598433e-01, -2.4952958e+00,
        1.7794682e+00,  2.6589274e+00,  3.2492065e+00, -2.8935118e+00,
        1.1031879e+00,  2.3222227e+00,  6.3222959e-03,  8.4244430e-01,
        2.8179293e+00,  1.7068143e+00,  3.5040204e-03, -1.4183084e+00,
       -2.3829527e+00, -1.6503531e+00, -2.4414184e+00, -1.5148463e+00,
        2.1881945e-02, -2.0980315e+00,  3.7607574e-01, -1.4977980e+00,
        4.3674059e+00,  7.6944888e-01, -5.6418532e-01, -1.0047307e+00,
        1.9735342e-01, -1.3633486e+00, -2.3349409e+00, -1.2895327e+00,
        7.9056460e-01, -8.7589353e-01, -1.3541988e+00, -1.4744925e+00,
       -3.9973000e-01,  2.4608760e+00,  9.8813611e-01,  5.7009113e-01,
        1.7797036e+00, -2.0908549e+00,  7.5216353e-01,  1.0540767e+00,
       -3.2858655e+00, -4.8938724e-01,  6.6385561e-01, -1.0152043e+00,
      

In [19]:
sims_g = model.wv.most_similar('guarantee', topn=15)
sims_g

[('guaranty', 0.7401164174079895),
 ('warrantee', 0.6916505098342896),
 ('promise', 0.6578118205070496),
 ('warranty', 0.6492654085159302),
 ('warrenty', 0.6484857201576233),
 ('honor', 0.6302893161773682),
 ('warranty-', 0.6143152117729187),
 ('guarantees', 0.5820949077606201),
 ('deny', 0.5750207901000977),
 ('guarentee', 0.5746412873268127),
 ('authorize', 0.5667845606803894),
 ('warranties', 0.5653486251831055),
 ('assure', 0.553169846534729),
 ('contract', 0.5524969100952148),
 ('offer', 0.5469974875450134)]

In [20]:
sims_g2 = model2.wv.most_similar('guarantee', topn=15)
sims_g2

[('warrantee', 0.678400456905365),
 ('warranty', 0.6528030037879944),
 ('honor', 0.6372912526130676),
 ('commitment', 0.6358263492584229),
 ('warranties', 0.6334655284881592),
 ('promise', 0.6179825663566589),
 ('guaranty', 0.6117643713951111),
 ('warrenty', 0.6103754639625549),
 ('deny', 0.60561603307724),
 ('commitments', 0.6052522659301758),
 ('guarantees', 0.5768384337425232),
 ('assure', 0.5741132497787476),
 ('authorize', 0.570785641670227),
 ('reimbursement', 0.5619900822639465),
 ('refunds', 0.560116708278656)]

In [21]:
sims_w = model.wv.most_similar('warranty', topn=15)
sims_w

[('warrenty', 0.9041443467140198),
 ('warrantee', 0.8994622230529785),
 ('warranties', 0.8032267093658447),
 ('warranty-', 0.6907905340194702),
 ('contract', 0.674579918384552),
 ('warantee', 0.6734037399291992),
 ('guarantee', 0.649265468120575),
 ('waranty', 0.6477544903755188),
 ('guaranty', 0.6402029395103455),
 ('squaretrade', 0.6314033269882202),
 ('policy', 0.6308709383010864),
 ('rebate', 0.617492139339447),
 ('return/exchange', 0.5849823951721191),
 ('warrantied', 0.5847527384757996),
 ('next-business-day', 0.5815829038619995)]

In [22]:
sims_w2 = model2.wv.most_similar('warranty', topn=15)
sims_w2

[('warrantee', 0.8888049125671387),
 ('warrenty', 0.8630625009536743),
 ('warranties', 0.8201990127563477),
 ('squaretrade', 0.6600282192230225),
 ('guarantee', 0.6528029441833496),
 ('policy', 0.6502243280410767),
 ('warantee', 0.6488725543022156),
 ('contract', 0.6460720896720886),
 ('warrantied', 0.6342207789421082),
 ('rebate', 0.6273413300514221),
 ('warranty-', 0.6011028289794922),
 ('refunds', 0.5996984243392944),
 ('receipt', 0.5903002619743347),
 ('waranty', 0.5849115252494812),
 ('reseller', 0.581851065158844)]

In [ ]:
train[(train.reviewText.str.find('warrenty') != -1) | (train.reviewText.str.find('guarantee') != -1)]

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
167,4,NaN,True,2016-08-04,A3IZBWYYS8J9YL,B000001OM4,{'Format:': ' Electronics'},Jeffrey G,This adapter works well with my 1994 Corvette....,Works well with 94 Vette,1470268800
484,5,NaN,True,2018-04-13,A2S2TX0PGZKD37,B00001P4ZH,{'Color:': ' Black/Silver'},hikingfoole_85,3rd pair of these I've purchased. My wife has...,Great sounding Koss Headphones,1523577600
534,5,NaN,False,2017-04-04,AYVUFVX51BGR9,B00001P4XA,{'Color:': ' Blue'},Daisey Love,The sound from these is AMAZING AMAZING AMAZIN...,AMAZING AMAZING SOUND!,1491264000
554,5,NaN,True,2017-01-04,A37KG5HXY9VWPW,B00001P4ZH,{'Color:': ' Black/Silver'},THE KING OF PANGAEA,A+++ Cans. I have been using these for portabl...,STILL A BARGAIN AFTER ALL THESE YEARS,1483488000
789,3,NaN,False,2016-01-22,A1DU6K1MJTVR7D,B00002EQCW,NaN,RailFan,I have 3 Netgear switches and 2 Netgear router...,"Good Quality, but usually only 3 effective add...",1453420800
...,...,...,...,...,...,...,...,...,...,...,...
836960,2,NaN,True,2017-12-06,A2101DHHDMZ93X,B01GEVLS1I,NaN,Just_Keith,I got this to use exclusively with my Roku Exp...,This wouldn't work for me with a Roku Express,1512518400
837251,3,NaN,False,2016-07-11,A2YXQMZ32BY8XH,B01GJWCA3M,{'Color:': ' 1pc - Black'},IAMM3Z,I would probably reverse engineer this stylus ...,Not For Me,1468195200
837706,5,2.0,False,2016-11-21,A2A0FBLKJN4ZY5,B01GUNVCS4,NaN,Amazon Customer,"Amazing laptop, I've owned Acers since about 1...",Acer is the best hands down. And best bang for...,1479686400
837961,5,NaN,True,2018-09-27,A1W34IXADXA6Z9,B01H14SFRM,{'Color:': ' 12 Outlets + 3 USB Black'},Gokudo,Purchased to protect a Pc gaming rig and recha...,Nice alternative to belkin. Cost effective for...,1538006400


In [ ]:
train[train.reviewText.str.contains('warrenty|guarantee')]

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
167,4,NaN,True,2016-08-04,A3IZBWYYS8J9YL,B000001OM4,{'Format:': ' Electronics'},Jeffrey G,This adapter works well with my 1994 Corvette....,Works well with 94 Vette,1470268800
484,5,NaN,True,2018-04-13,A2S2TX0PGZKD37,B00001P4ZH,{'Color:': ' Black/Silver'},hikingfoole_85,3rd pair of these I've purchased. My wife has...,Great sounding Koss Headphones,1523577600
534,5,NaN,False,2017-04-04,AYVUFVX51BGR9,B00001P4XA,{'Color:': ' Blue'},Daisey Love,The sound from these is AMAZING AMAZING AMAZIN...,AMAZING AMAZING SOUND!,1491264000
554,5,NaN,True,2017-01-04,A37KG5HXY9VWPW,B00001P4ZH,{'Color:': ' Black/Silver'},THE KING OF PANGAEA,A+++ Cans. I have been using these for portabl...,STILL A BARGAIN AFTER ALL THESE YEARS,1483488000
789,3,NaN,False,2016-01-22,A1DU6K1MJTVR7D,B00002EQCW,NaN,RailFan,I have 3 Netgear switches and 2 Netgear router...,"Good Quality, but usually only 3 effective add...",1453420800
...,...,...,...,...,...,...,...,...,...,...,...
836960,2,NaN,True,2017-12-06,A2101DHHDMZ93X,B01GEVLS1I,NaN,Just_Keith,I got this to use exclusively with my Roku Exp...,This wouldn't work for me with a Roku Express,1512518400
837251,3,NaN,False,2016-07-11,A2YXQMZ32BY8XH,B01GJWCA3M,{'Color:': ' 1pc - Black'},IAMM3Z,I would probably reverse engineer this stylus ...,Not For Me,1468195200
837706,5,2.0,False,2016-11-21,A2A0FBLKJN4ZY5,B01GUNVCS4,NaN,Amazon Customer,"Amazing laptop, I've owned Acers since about 1...",Acer is the best hands down. And best bang for...,1479686400
837961,5,NaN,True,2018-09-27,A1W34IXADXA6Z9,B01H14SFRM,{'Color:': ' 12 Outlets + 3 USB Black'},Gokudo,Purchased to protect a Pc gaming rig and recha...,Nice alternative to belkin. Cost effective for...,1538006400


In [25]:
keywords = ['guarantee', 'warranty']

for i in sims_g:
  keywords.append(i[0])

for j in sims_w:
  keywords.append(j[0])


In [26]:
keywords

['guarantee',
 'warranty',
 'guaranty',
 'warrantee',
 'promise',
 'warranty',
 'warrenty',
 'honor',
 'warranty-',
 'guarantees',
 'deny',
 'guarentee',
 'authorize',
 'warranties',
 'assure',
 'contract',
 'offer',
 'warrenty',
 'warrantee',
 'warranties',
 'warranty-',
 'contract',
 'warantee',
 'guarantee',
 'waranty',
 'guaranty',
 'squaretrade',
 'policy',
 'rebate',
 'return/exchange',
 'warrantied',
 'next-business-day']

In [27]:
keywords = list(set(keywords))

In [28]:
keywords

['guarantees',
 'policy',
 'promise',
 'deny',
 'next-business-day',
 'honor',
 'rebate',
 'warranty-',
 'warranty',
 'guarantee',
 'warantee',
 'waranty',
 'authorize',
 'guarentee',
 'warrantee',
 'contract',
 'warrantied',
 'squaretrade',
 'return/exchange',
 'assure',
 'guaranty',
 'warrenty',
 'offer',
 'warranties']

In [29]:
keywords.remove('promise')
keywords.remove('honor')
keywords.remove('deny')
keywords.remove('authorize')
keywords.remove('contract')
keywords.remove('offer')
keywords.remove('squaretrade')
keywords.remove('rebate')
keywords.remove('next-business-day')

keywords

['guarantees',
 'policy',
 'warranty-',
 'warranty',
 'guarantee',
 'warantee',
 'waranty',
 'guarentee',
 'warrantee',
 'warrantied',
 'return/exchange',
 'assure',
 'guaranty',
 'warrenty',
 'warranties']

In [32]:
train_guarantee = train[train.reviewText.str.contains('|'.join(keywords))]

In [33]:
train_guarantee.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20560 entries, 167 to 838901
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   overall         20560 non-null  int64 
 1   vote            7286 non-null   object
 2   verified        20560 non-null  bool  
 3   reviewTime      20560 non-null  object
 4   reviewerID      20560 non-null  object
 5   asin            20560 non-null  object
 6   style           12131 non-null  object
 7   reviewerName    20555 non-null  object
 8   reviewText      20560 non-null  object
 9   summary         20559 non-null  object
 10  unixReviewTime  20560 non-null  int64 
dtypes: bool(1), int64(2), object(8)
memory usage: 1.7+ MB


In [34]:
product_overall_for_guarantee = train_guarantee.groupby(['asin']).agg({'overall':'mean'})
product_overall_for_guarantee

,overall
asin,
6541654530,1.0
9800466657,5.0
B000001OM4,4.0
B00000J4EY,3.0
B00000JPPI,1.0
...,...
B01HIS30OY,3.0
B01HIS5N3K,3.0
B01HIURQWE,5.0
